In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import tkinter as tk
from tkinter import filedialog, messagebox
from pathlib import Path

def select_file(title, file_types, save=False):
    """Unified file selection function"""
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    root.attributes('-topmost', True)  # Keep dialog on top
    
    try:
        if save:
            file_path = filedialog.asksaveasfilename(
                title=title,
                filetypes=file_types,
                defaultextension=file_types[0][1]
            )
        else:
            file_path = filedialog.askopenfilename(
                title=title,
                filetypes=file_types
            )
    finally:
        root.destroy()
    
    return file_path if file_path else None

def optimize_dataframe(df):
    """Optimize DataFrame memory usage by converting to appropriate dtypes"""
    
    # Convert date columns to datetime using efficient parsing
    date_columns = ['DOB', 'TRAVEL DATE', 'RETURN DATE']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format='mixed', errors='coerce')
    
    # Convert categorical columns to category dtype
    categorical_columns = ['GENDER', 'MARITAL STATUS', 'NATIONALITY', 
                         'PURPOSE OF VISIT', 'CARRIER TYPE', 'CARRIER NAME',
                         'EMBARK PORT']
    for col in categorical_columns:
        if col in df.columns:
            df[col] = df[col].astype('category')
    
    return df

def calculate_age_vectorized(dob_series):
    """Calculate age using vectorized operations"""
    today = pd.Timestamp.now()
    return ((today - dob_series).dt.days / 365.25)

def create_table1(df):
    """Create Table 1 from a pandas DataFrame with optimized operations"""
    results = []
    total_n = len(df)
    
    # Calculate age efficiently
    print("Calculating age statistics...")
    df['AGE'] = calculate_age_vectorized(df['DOB'])
    age_stats = df['AGE'].agg(['mean', 'std'])
    results.append({
        'Category': 'Demographics',
        'Characteristic': 'Age',
        'Value': f"{age_stats['mean']:.1f} ± {age_stats['std']:.1f}",
        'N': total_n
    })
    
    # Process categorical variables efficiently
    categorical_vars = {
        'Demographics': ['GENDER', 'MARITAL STATUS', 'NATIONALITY'],
        'Travel': ['PURPOSE OF VISIT', 'CARRIER TYPE', 'CARRIER NAME', 'EMBARK PORT']
    }
    
    print("Processing categorical variables...")
    for category, variables in categorical_vars.items():
        for var in variables:
            # Determine how many top values to show
            if var in ['NATIONALITY']:
                n_top = 10
            elif var in ['CARRIER NAME', 'EMBARK PORT']:
                n_top = 3
            else:
                n_top = None
                
            value_counts = df[var].value_counts()
            if n_top:
                value_counts = value_counts.head(n_top)
                
            percentages = (value_counts / total_n * 100)
            
            # Add header for variables where we're only showing top N
            if n_top:
                results.append({
                    'Category': category,
                    'Characteristic': f"{var.title().replace('_', ' ')} (Top {n_top})",
                    'Value': '',
                    'N': '',
                    'Percentage': ''
                })
            
            for val, count in value_counts.items():
                results.append({
                    'Category': category,
                    'Characteristic': var.title().replace('_', ' ') if not n_top else "   " + str(val),
                    'Value': val if not n_top else '',
                    'N': count,
                    'Percentage': f"{percentages[val]:.1f}%"
                })
    
    # Calculate length of stay efficiently
    print("Calculating length of stay...")
    df['LENGTH_OF_STAY'] = (df['RETURN DATE'] - df['TRAVEL DATE']).dt.days
    stay_stats = df['LENGTH_OF_STAY'].agg(['mean', 'std'])
    results.append({
        'Category': 'Travel',
        'Characteristic': 'Length of Stay (days)',
        'Value': f"{stay_stats['mean']:.1f} ± {stay_stats['std']:.1f}",
        'N': df['LENGTH_OF_STAY'].notna().sum()
    })
    
    return pd.DataFrame(results)

def show_message(title, message, error=False):
    """Show message dialog"""
    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)
    try:
        if error:
            messagebox.showerror(title, message)
        else:
            messagebox.showinfo(title, message)
    finally:
        root.destroy()

def main():
    try:
        # File type definitions
        excel_types = [
            ('Excel files', '*.xlsx *.xls'),
            ('All files', '*.*')
        ]
        
        # Get input file
        input_file = select_file(
            title='Select Excel Data File',
            file_types=excel_types
        )
        
        if not input_file:
            print("No file selected. Exiting.")
            return
        
        # Read the data with optimized settings
        print("Reading data...")
        df = pd.read_excel(
            input_file,
            # Only read needed columns
            usecols=['DOB', 'GENDER', 'MARITAL STATUS', 'NATIONALITY',
                    'PURPOSE OF VISIT', 'CARRIER TYPE', 'CARRIER NAME', 
                    'EMBARK PORT', 'TRAVEL DATE', 'RETURN DATE']
        )
        
        # Optimize DataFrame
        print("Optimizing data structure...")
        df = optimize_dataframe(df)
        
        # Create Table 1
        print("Generating Table 1...")
        table1_df = create_table1(df)
        
        # Get output file
        save_types = [
            ('Excel files', '*.xlsx'),
            ('CSV files', '*.csv'),
            ('All files', '*.*')
        ]
        
        output_file = select_file(
            title='Save Table 1 As',
            file_types=save_types,
            save=True
        )
        
        if not output_file:
            print("No output location selected. Exiting.")
            return
        
        # Ensure proper file extension
        if not output_file.endswith(('.xlsx', '.csv')):
            output_file += '.xlsx'
        
        # Save based on extension
        print("Saving results...")
        if output_file.endswith('.xlsx'):
            with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
                table1_df.to_excel(writer, sheet_name='Table 1', index=False)
                
                # Auto-adjust column widths
                worksheet = writer.sheets['Table 1']
                for idx, col in enumerate(table1_df.columns):
                    max_length = max(
                        table1_df[col].astype(str).apply(len).max(),
                        len(col)
                    )
                    worksheet.column_dimensions[chr(65 + idx)].width = max_length + 2
        else:
            table1_df.to_csv(output_file, index=False)
        
        print(f"Table 1 has been saved to: {output_file}")
        show_message("Success", f"Table 1 has been successfully created and saved to:\n{output_file}")
        
    except Exception as e:
        error_msg = f"An error occurred:\n{str(e)}"
        print(error_msg)
        show_message("Error", error_msg, error=True)

if __name__ == "__main__":
    main()

Reading data...
Optimizing data structure...
Generating Table 1...
Calculating age statistics...
Processing categorical variables...
Calculating length of stay...
Saving results...
Table 1 has been saved to: C:/Users/janai/Documents/Tourism Files/Tourism Data/Table 1 2024 Arrivals.xlsx
